# SIR Simulation

## Name: Rohit Dev

In [ ]:
# Import libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import missingno as msno
import plotly.graph_objects as go

from datetime import datetime

In [ ]:
%matplotlib inline
sns.set(style='darkgrid')
mpl.rcParams['figure.figsize'] = (16,9)
pd.set_option('display.max_rows', 500)

In [ ]:
# SIR Model is being for shown 3 countries
country_list = ['Germany', 'India', 'Spain']

In [ ]:
# Import raw data
raw_data=pd.read_csv('../data/raw/owid-covid-data.csv')

In [ ]:
# display raw data
raw_data.head()

In [ ]:
# Store the country list
country_list = raw_data['location'].unique()

In [ ]:
# Import the processed data
data_analysis = pd.read_csv('../data/processed/processed_data.csv')
data_analysis.set_index(['date'], inplace=True)

In [ ]:
# Sort the infected cases for all countries in separate data frame
data_infected= pd.DataFrame(data_analysis.index)
data_infected.set_index(['date'], inplace=True)

In [ ]:
for each in country_list:
    data_infected[each] = pd.DataFrame(data_analysis[each])

In [ ]:
data_infected.tail()

In [ ]:
# Parameters

N0 = 83408554.0 # Population size Germany
beta = 0.4
gamma = 0.1  # 1(gamma) Person can infect 4(beta) people

I0 = data_infected.Germany[100]   # Initial infected size
S0 = N0-I0  # Population prone to infection
R0 = 0  # Initial recovered size

In [ ]:
# SIR Function
def SIR_model(SIR, beta, gamma):
    ''' Simple SIR model
        S: susceptible population
        I: infected people
        R: recovered people
        beta:
        
        overall condition is that the sum of changes (differences) sum up to 0
        dS+dI+dR=0
        S+I+R = N (constant size of population)
    '''
    
    S,I,R = SIR
    dS_dt = -beta*S*I/N0
    dI_dt = beta*S*I/N0-gamma*I
    dR_dt = gamma*I
    return ([dS_dt, dI_dt, dR_dt])

In [ ]:
SIR = np.array([S0,I0,R0])
propagation_rates = pd.DataFrame(columns={'susceptible':S0, 'infected':I0, 'recovered':R0})

for each_t in np.arange(300): # calculation for 300 days
#for each_t in np.arange(len(data_infected)):
    new_delta_vec = SIR_model(SIR, beta, gamma)
    SIR = SIR+new_delta_vec
    propagation_rates = propagation_rates.append({'susceptible':SIR[0], 'infected':SIR[1], 'recovered':SIR[2]}, ignore_index=True)

In [ ]:
propagation_rates # For Germany

In [ ]:
fig, ax1 = plt.subplots(1,1)

ax1.plot(propagation_rates.index, propagation_rates.infected, label = 'infected', color = 'red')
ax1.plot(propagation_rates.index, propagation_rates.susceptible, label = 'susceptible', color = 'green')
ax1.plot(propagation_rates.index, propagation_rates.recovered, label = 'recovered', color = 'blue')


ax1.set_yscale('linear')

ax1.set_title('Scenario SIR simulations for Germany', size = 16)
ax1.set_xlabel('time in days', size = 16)
ax1.legend(loc = 'best', prop={'size':16})

# End